In [3]:
#importing libraries
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np

In [4]:


#reading camera feed
cap = cv2.VideoCapture(0)
#loading haar casscase
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#loading caffe models
ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']


while True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    #loping over face coordinates
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        detected_face = img[y:y+h, x:x+w]
        
        lab_img= cv2.cvtColor(detected_face, cv2.COLOR_BGR2LAB)

        #Splitting the LAB image to L, A and B channels, respectively
        l, a, b = cv2.split(lab_img)


        #Apply histogram equalization to the L channel
        equ = cv2.equalizeHist(l)

        #Combine the Hist. equalized L-channel back with A and B channels
        updated_lab_img1 = cv2.merge((equ,a,b))

        #Convert LAB image back to color (RGB)
        hist_eq_img = cv2.cvtColor(updated_lab_img1, cv2.COLOR_LAB2BGR)
        cv2.imshow('histogram equalization img', hist_eq_img)
        

        #clahe equalization
        #Apply CLAHE to L channel
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        clahe_img = clahe.apply(l)
        

        #Combine the CLAHE enhanced L-channel back with A and B channels
        updated_lab_img2 = cv2.merge((clahe_img,a,b))

        #Convert LAB image back to color (RGB)
        CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2BGR)
        cv2.imshow('CLAHE img', CLAHE_img)
        
        #inputting clahe image for preprocessing
        detected_face_blob = cv2.dnn.blobFromImage(CLAHE_img, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        
        #detections using caffe.qq
        genderNet.setInput(detected_face_blob)
        ageNet.setInput(detected_face_blob)
        
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        
        
        agePreds=ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        
        
        label = "{}{}".format(gender,age)
         
        cv2.putText(img, label, (x+30, y+30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        print("Age",age," ","gender",gender)
    cv2.imshow('ORGINAL img', img)
    
    #press q to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Age (8-12)   gender Male
Age (38-43)   gender Male
Age (38-43)   gender Male
Age (38-43)   gender Male
Age (4-6)   gender Male
Age (4-6)   gender Male
Age (8-12)   gender Female
Age (4-6)   gender Male
Age (8-12)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (8-12)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (8-12)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender Male
Age (25-32)   gender 